# Решение СДУ методом Рунге-Кутта

## Теоретическая часть

Пусть требуется решить систему дифференциальных уравнений первого порядка:
$$
\left\{\begin{array}{l}
\frac{d y_1}{d x}=y_2 \\
\frac{d y_2}{d x}=e^{-x \cdot y_1}
\end{array}\right.
$$

методом Рунге-Кутта на отрезке $[0,1]$ с шагом $h=0.1$. Начальные условия: $x_0=0 ; y_1(0)=0 ; y_2(0)=0$.
<!-- Воспользуемся формулой (5) и запишем выражения для $y_{i, 1}$ и $y_{i, 2}$ :
$$
\left\{\begin{aligned}
y_{i, 1} & =y_{(i-1), 1}+0.1 \cdot y_{(i-1), 2} \\
y_{i, 2} & =y_{(i-1), 2}+0.1 \cdot e^{-x_{i-1} \cdot y_{(i-1), 1}} \\
x_i & =x_{i-1}+h
\end{aligned}\right.
$$ -->

При использовании метода Рунге-Кутты, расчетные формулы примут следующий вид:
$$
\left\{\begin{aligned}
y_{i, 1} & =y_{(i-1), 1}+h / 6 \cdot\left(k_{1,1}+2 \cdot k_{2,1}+2 \cdot k_{3,1}+k_{4,1}\right) \\
y_{i, 2} & =y_{(i-1), 2}+h / 6 \cdot\left(k_{1,2}+2 \cdot k_{2,2}+2 \cdot k_{3,2}+k_{4,2}\right) \\
x_i & =x_{i-1}+h
\end{aligned}\right. \quad \quad \quad \quad \quad \quad \quad  (8)
$$

где
$$
\begin{aligned}
& k_{\mathbf{1}, \mathbf{1}}=f_{\mathbf{1}}\left(x, y_{(i-1), \mathbf{1}}, y_{(i-1), \mathbf{2}}\right) ; \\
& k_{\mathbf{1}, \mathbf{2}}=f_{\mathbf{2}}\left(x, y_{(i-1), \mathbf{1}}, y_{(i-1), 2}\right) ; \\
& k_{\mathbf{2}, \mathbf{1}}=f_{\mathbf{1}}\left(x+\frac{h}{2}, y_{(i-\mathbf{1}), \mathbf{1}}+k_{\mathbf{1}, \mathbf{1}} \cdot \frac{h}{2}, y_{(i-\mathbf{1}), \mathbf{2}}+k_{\mathbf{1}, \mathbf{2}} \cdot \frac{h}{2}\right) ; \\
& k_{\mathbf{2}, \mathbf{2}}=f_{\mathbf{2}}\left(x+\frac{h}{2}, y_{(i-1), \mathbf{1}}+k_{\mathbf{1}, \mathbf{1}} \cdot \frac{h}{2}, y_{(i-\mathbf{1}), \mathbf{2}}+k_{\mathbf{1}, \mathbf{2}} \cdot \frac{h}{2}\right) ; \\
& k_{3,1}=f_1\left(x+\frac{h}{2}, y_{(i-1), 1}+k_{2,1} \cdot \frac{h}{2}, y_{(i-1), 2}+k_{2,2} \cdot \frac{h}{2}\right) \text {; } \\
& k_{3,2}=f_2\left(x+\frac{h}{2}, y_{(i-1), 1}+k_{2,1} \cdot \frac{h}{2}, y_{(i-1), 2}+k_{2,2} \cdot \frac{h}{2}\right) \text {; } \\
& k_{\mathbf{4}, \mathbf{1}}=f_{\mathbf{1}}\left(x+h, y_{(i-\mathbf{1}), \mathbf{1}}+k_{\mathbf{3}, \mathbf{1}} \cdot h, y_{(i-\mathbf{1}), \mathbf{2}}+k_{\mathbf{3}, \mathbf{2}} \cdot h\right) \text {; } \\
& k_{\mathbf{4}, \mathbf{2}}=f_{\mathbf{2}}\left(x+h, y_{(i-1), \mathbf{1}}+k_{\mathbf{3}, \mathbf{1}} \cdot h, y_{(i-\mathbf{1}), \mathbf{2}}+k_{\mathbf{3}, \mathbf{2}} \cdot h\right) \text {. } \\
&
\end{aligned} \quad \quad \quad \quad \quad \quad \quad  (9)
$$

где $h$ - шаг интегрирования; $f_1\left(x_i, y_{(i-1), 1}, y_{(i-1), 2}\right)$ и $f_2\left(x_i, y_{(i-1), 1}, y_{(i-1), 2}\right)$ - правые части дифференциальных уравнений, $k_{1, j}, k_{2, j}, k_{3, j}, k_{4, j}$ - параметры метода Рунге-Кутты для $j$-го уравнения.

<!-- Решим методом Рунге-Кутты пример, приведенный на слайде 7. Воспользуемся формулами (8), (9) и запишем выражения для нахождения значений искомых переменных $y_{i, 1}$ и $y_{i, 2}$ :
\begin{aligned}
k_{1,1}& 1=y_{(i-1),2};\quad\quad\quad k_{1,2}=\exp\left(-x_i\cdot y_{(i-1),1}\right);  \\
k_{2,}&   1=y_{(i-1),2}+k_{1,2}\cdot\frac h2;\quad k_{2,2}=\exp\left[-\left(x_i+\frac h2\right)\cdot\left(y_{(i-1),1}+k_{1,1}\cdot\frac h2\right)\right.  \\
k_{3,1}& =y_{(i-1),2}+k_{2,2}\cdot\frac h2;\quad k_{3,2}=\exp\left[-\left(x_i+\frac h2\right)\cdot\left(y_{(i-1),1}+k_{2,1}\cdot\frac h2\right)\right]  \\
\text{k4,1}& =y_{(i-1),2}+k_{3,2}\cdot h;\quad k_{4,2}=\exp\left[-(x_i+h)\cdot(y_{(i-1),1}+k_{3,1}\cdot h)\right]  \\
&\begin{cases}y_{i,1}=y_{(i-1),1}+\frac{0.1}6\cdot(k_{1,1}+2\cdot k_{2,1}+2\cdot k_{3,1}+k_{4,1})\\y_{i,2}=y_{(i-1),2}+\frac{0.1}6\cdot(k_{1,2}+2\cdot k_{2,2}+2\cdot k_{3,2}+k_{4,2})\\x_i=x_{i-1}+0.1&\end{cases}
\end{aligned} -->

Решим методом Рунге-Кутты пример. Воспользуемся формулами (8), (9) и запишем выражения для нахождения значений искомых переменных $y_{i, 1}$ и $y_{i, 2}$ :
$$
\begin{aligned}
& k_{1,1}=y_{(i-1), 2} ; \\
& k_{1,2}=\exp \left(-x_i \cdot y_{(i-1), 1}\right) \text {; } \\
& k_{2,1}=y_{(i-1), 2}+k_{1,2} \cdot \frac{h}{2} \text {; } \\
& k_{2,2}=\exp \left[-\left(x_i+\frac{h}{2}\right) \cdot\left(y_{(i-1), 1}+k_{1,1} \cdot \frac{h}{2}\right)\right] \\
& k_{3,1}=y_{(i-1), 2}+k_{2,2} \cdot \frac{h}{2} \text {; } \\
& k_{3,2}=\exp \left[-\left(x_i+\frac{h}{2}\right) \cdot\left(y_{(i-1), 1}+k_{2,1} \cdot \frac{h}{2}\right)\right] \\
& k_{4,1}=y_{(i-1), 2}+k_{3,2} \cdot h \text {; } \\
& k_{4,2}=\exp \left[-\left(x_i+h\right) \cdot\left(y_{(i-1), 1}+k_{3,1} \cdot h\right)\right] \\
& \left\{\begin{aligned}
y_{i, 1} & =y_{(i-1), 1}+\frac{0.1}{6} \cdot\left(k_{1,1}+2 \cdot k_{2,1}+2 \cdot k_{3,1}+k_{4,1}\right) \\
y_{i, 2} & =y_{(i-1), 2}+\frac{0.1}{6} \cdot\left(k_{1,2}+2 \cdot k_{2,2}+2 \cdot k_{3,2}+k_{4,2}\right) \\
x_i & =x_{i-1}+0.1
\end{aligned}\right. \\
&
\end{aligned}
$$

In [2]:
import math


def equations(x , y ) :  
    """Функция, содержащая правые части дифф. уравнений"""
    return [y[1], math.exp(-x * y[0])]


def rk(func, x0, xf, y0, h) :
    count = int (( xf - x0 ) / h ) + 1
    y = [ y0[:] ]
    x = x0
    for i in range (1 , count ) :
        k1 = func(x, y[i - 1])
        k2 = func (x + h/2, list(map(lambda arr1, arr2: arr1 + arr2 * h/2, y[i - 1], k1)))
        k3 = func (x + h/2, list(map(lambda arr1, arr2: arr1 + arr2 * h/2, y[i - 1], k2)))
        k4 = func (x + h, list(map(lambda arr1, arr2: arr1 + arr2 * h, y[i - 1], k3)))
        
        y . append ([])
        
        for j in range ( len ( y0 ) ) :
            y[i].append(y[i - 1][j] + h/6 * (k1[j] + 2 * k2[j] + 2 * k3[j] + k4[j]))

        x += h
    return y

In [11]:
from icecream import ic 

ic()
for l in rk(equations, 0, 1, [0, 0], 0.1):
    # print(l)
    ic(l)
    # ic()

ic| 979194122.py:3 in <module> at 22:27:40.479
ic| l: [0, 0]
ic| l: [0.004999791679686959, 0.09998750234339197]
ic| l: [0.019992089353337197, 0.19980027824237273]
ic| l: [0.04493954532954178, 0.2989921821997826]
ic| l: [0.07974589273138522, 0.39683477618392093]
ic| l: [0.1242292261307227, 0.49235154280802335]
ic| l: [0.1781000081292174, 0.5843789596377397]
ic| l: [0.24094662432104696, 0.67165612248553]
ic| l: [0.3122311354618596, 0.7529375201538153]
ic| l: [0.39129695254854, 0.8271160064996047]
ic| l: [0.4773885589403407, 0.8933374434985747]
